# ACC RAG Chain LLM

In [1]:
import os
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader

from src.fs import read_text_dir

## Read ACR Conditions

In [2]:
# Read all custom ACR appropriateness criteria documents
docs_raw = read_text_dir("data/conditions")
docs_text = "\n\n---\n\n".join(docs_raw.values())

## Indexing

### Split Documents

Split by markdown headings

In [30]:
# Determine levels of headings to be splitted
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, 
                                               strip_headers = False)
docs_splited = markdown_splitter.split_text(docs_text)

In [32]:
docs_splited[0:2]

[Document(metadata={'Header 1': 'Abnormal Liver Function Tests'}, page_content='# Abnormal Liver Function Tests  \nAmerican College of Radiology: ACR Appropriateness Criteria®'),
 Document(metadata={'Header 1': 'Abnormal Liver Function Tests', 'Header 3': 'Variant 1: **Abnormal liver function tests. Hepatocellular predominance with mild aminotransferase increase. Initial imaging.**'}, page_content='### Variant 1: **Abnormal liver function tests. Hepatocellular predominance with mild aminotransferase increase. Initial imaging.**  \n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| US abdomen                                 | Usually Appropriate            | O                        |\n| US duplex Doppler abdomen                  | Usually Appropriate            | O                        |\n| US shear wave elastography ab

### Create Vector Store

In [33]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(docs_splited, OpenAIEmbeddings())

Test Similarity search

In [35]:
query = "Abnormal LFT"
docs = db.similarity_search(query, k = 3)
print(docs)

[Document(metadata={'Header 1': 'Abnormal Liver Function Tests', 'Header 3': 'Variant 3: **Abnormal liver function tests. Cholestatic predominance. Elevated alkaline phosphatase with or without elevated gamma-glutamyl transpeptidase. Initial imaging.**'}, page_content='### Variant 3: **Abnormal liver function tests. Cholestatic predominance. Elevated alkaline phosphatase with or without elevated gamma-glutamyl transpeptidase. Initial imaging.**  \n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| US abdomen                                 | Usually Appropriate            | O                        |\n| MRI abdomen without and with IV contrast with MRCP | Usually Appropriate         | O                        |\n| CT abdomen and pelvis with IV contrast     | Usually Appropriate            | ⚫⚫⚫                    |\n| US 

### Create Retriever

In [43]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4},
)

## Prompt Template

In [44]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

system_prompt = """You are an clinical decision support assistance for radiologist. 
Your task is to determine the imaging appropriateness from the input patient case, according to the provided context from ACR Appropriateness Criteria.
"""

human_prompt = """
Case: 

{case}

Context: 
Use the provided context only to answer the question. If you don't know the answer, just say that you don't know.

{context}

Questions: Is imaging typically appropriate for this case? If so, please specify the most suitable imaging modality and whether a contrast agent is required. Exclude 'May Be Appropriate' and 'Usually Not Appropriate' as potential responses.
"""

prompt_template = ChatPromptTemplate([
    ("system", system_prompt),
    ("human", human_prompt)
])

# Test prompt template
print(prompt_template.invoke({"case": "Test Case", 
                              "context": "test context"}).to_string())



System: You are an clinical decision support assistance for radiologist. 
Your task is to determine the imaging appropriateness from the input patient case, according to the provided context from ACR Appropriateness Criteria.

Human: 
Case: 

Test Case

Context: 
Use the provided context only to answer the question. If you don't know the answer, just say that you don't know.

test context

Questions: Is imaging typically appropriate for this case? If so, please specify the most suitable imaging modality and whether a contrast agent is required. Exclude 'May Be Appropriate' and 'Usually Not Appropriate' as potential responses.



## Final Chain

In [45]:
llm = ChatOpenAI(model="gpt-4o-mini")
rag_chain = {"context": retriever, 
             "case": RunnablePassthrough()} | prompt_template | llm

## Execute 🚀

### RLQ Abdominal Pain

In [46]:
case_1 = """
Patient age: 15 years old
Sex: Female
Chief Complaint: Right lower quadrant abdominal pain
The patient presents with a 2-day history of progressively worsening right lower quadrant abdominal pain. 
She reports loss of appetite but denies nausea or vomiting. No fever or diarrhea. Her last menstrual period was 2 weeks ago. 
She denies any history of similar pain episodes and has no significant past medical history. 
"""

response_1 = rag_chain.invoke(case_1)
print(response_1.content)

Yes, imaging is typically appropriate for this case. The most suitable imaging modality is a CT abdomen and pelvis with IV contrast, which is categorized as "Usually Appropriate."


In [47]:
retriever.invoke(case_1)

[Document(metadata={'Header 1': 'Right Lower Quadrant Pain', 'Header 3': 'Variant 1: **Right lower quadrant pain. Initial imaging.**'}, page_content='### Variant 1: **Right lower quadrant pain. Initial imaging.**  \n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| CT abdomen and pelvis with IV contrast     | Usually Appropriate            | ⚫⚫⚫                    |\n| US abdomen                                 | May Be Appropriate             | O                        |\n| US pelvis                                  | May Be Appropriate             | O                        |\n| MRI abdomen and pelvis without and with IV contrast | May Be Appropriate         | O                        |\n| MRI abdomen and pelvis without IV contrast | May Be Appropriate             | O                        |\n| CT abdomen and pelvis w

### Mesenteric Ischemia

In [49]:
case_2 = """
Patient age: 72 years old
Sex: Female
Chief Complaint: Severe abdominal pain and vomiting

The patient presents with sudden onset of severe, diffuse abdominal pain that started 6 hours ago, accompanied by multiple episodes of vomiting. She has a history of atrial fibrillation on warfarin, hypertension, and peripheral vascular disease. She reports the pain is out of proportion to physical examination findings.
"""

response_2 = rag_chain.invoke(case_2)
print(response_2.content)

In this case, imaging is typically appropriate. The most suitable imaging modality is a CT abdomen and pelvis with IV contrast, which is categorized as "Usually Appropriate."


In [48]:
retriever.invoke(case_2) 

[Document(metadata={'Header 1': 'Right Lower Quadrant Pain', 'Header 3': 'Variant 1: **Right lower quadrant pain. Initial imaging.**'}, page_content='### Variant 1: **Right lower quadrant pain. Initial imaging.**  \n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| CT abdomen and pelvis with IV contrast     | Usually Appropriate            | ⚫⚫⚫                    |\n| US abdomen                                 | May Be Appropriate             | O                        |\n| US pelvis                                  | May Be Appropriate             | O                        |\n| MRI abdomen and pelvis without and with IV contrast | May Be Appropriate         | O                        |\n| MRI abdomen and pelvis without IV contrast | May Be Appropriate             | O                        |\n| CT abdomen and pelvis w

### No Avilable Context

In [50]:
case_3 = """
Patient age: 30 years old 
Sex: Male. 
Chief complaint: Indeterminate palpable breast mass on the left side. 

A 2 cm hard mass is palpated in the left breast without nipple discharge or retraction, and no axillary lymphadenopathy. Mammography was indeterminant. There is no history of breast cancer in the family.
"""

response_3 = rag_chain.invoke(case_3)
print(response_3.content)

I don't know.
